In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import requests
import folium
import geopandas as gpd
from sqlalchemy import create_engine
from shapely.geometry import Point
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap
from geopy.distance import geodesic 

In [ ]:
#read in weather data

In [15]:
weather = pd.read_csv('../Data/Nashville_Weather.csv')


In [ ]:
#read in accident data

In [ ]:
accidents = pd.read_csv('../Data/Traffic_Accidents.csv')

In [3]:
accidents.head()

,Accident Number,Date and Time,Number of Motor Vehicles,Number of Injuries,Number of Fatalities,Property Damage,Hit and Run,Reporting Officer,Collision Type Code,Collision Type Description,...,Harmful Description,Street Address,City,State,ZIP,RPA,Precinct,Latitude,Longitude,Mapped Location
0,20170330208,04/14/2017 10:44:00 AM,2.0,1.0,0,NaN,False,716886,11.0,Front to Rear,...,MOTOR VEHICLE IN TRANSPORT,I24E & I24 E ENT RAMP,NASHVILLE,TN,37210.0,8102.0,SOUTH,36.1363,-86.7270,POINT (-86.727 36.1363)
1,20161069361,12/02/2016 08:09:00 PM,2.0,0.0,0,NaN,True,240979,4.0,ANGLE,...,MOTOR VEHICLE IN TRANSPORT,HAYWOOD LN & ANTIOCH PKE,NASHVILLE,TN,37211.0,8851.0,SOUTH,36.0702,-86.6848,POINT (-86.6848 36.0702)
2,20170141034,02/13/2017 12:56:00 PM,2.0,1.0,0,NaN,False,594473,4.0,ANGLE,...,MOTOR VEHICLE IN TRANSPORT,BLACKMAN RD & TIMBERHILL DR,NASHVILLE,TN,37220.0,8613.0,MIDTOW,36.0747,-86.7548,POINT (-86.7548 36.0747)
3,20161023575,11/17/2016 04:29:00 PM,2.0,0.0,0,NaN,False,909486,11.0,Front to Rear,...,MOTOR VEHICLE IN TRANSPORT,OLD HICKORY BLVD & BRENTWOOD COMMONS WAY,BRENTWOOD,TN,37027.0,8623.0,MIDTOW,36.0412,-86.7766,POINT (-86.7766 36.0412)
4,20170321614,04/11/2017 11:17:00 PM,2.0,0.0,0,NaN,False,110096,11.0,Front to Rear,...,MOTOR VEHICLE IN TRANSPORT,MT VIEW RD & BELL RD,ANTIOCH,TN,37013.0,8865.0,SOUTH,36.0500,-86.6496,POINT (-86.6496 36.05)


In [4]:
accidents.dtypes

Accident Number                object
Date and Time                  object
Number of Motor Vehicles      float64
Number of Injuries            float64
Number of Fatalities            int64
Property Damage                object
Hit and Run                    object
Reporting Officer              object
Collision Type Code           float64
Collision Type Description     object
Weather Code                  float64
Weather Description            object
Illumination Code             float64
Illumination Description       object
Harmful Code                   object
Harmful Description            object
Street Address                 object
City                           object
State                          object
ZIP                           float64
RPA                           float64
Precinct                       object
Latitude                      float64
Longitude                     float64
Mapped Location                object
dtype: object

In [5]:
accidents['Date and Time'] = pd.to_datetime(accidents['Date and Time'])
accidents.drop(columns=['Mapped Location'])

In [6]:
print(accidents['Date and Time'].min())
print(accidents['Date and Time'].max())


2013-01-01 00:15:00
2020-12-02 21:30:00


In [45]:
accidents.columns = accidents.columns.str.replace(' ', '')

In [47]:
accidents.dtypes

AccidentNumber                      object
DateandTime                 datetime64[ns]
NumberofMotorVehicles              float64
NumberofInjuries                   float64
NumberofFatalities                   int64
PropertyDamage                      object
HitandRun                           object
ReportingOfficer                    object
CollisionTypeCode                  float64
CollisionTypeDescription            object
WeatherCode                        float64
WeatherDescription                  object
IlluminationCode                   float64
IlluminationDescription             object
HarmfulCode                         object
HarmfulDescription                  object
StreetAddress                       object
City                                object
State                               object
ZIP                                float64
RPA                                float64
Precinct                            object
Latitude                           float64
Longitude  

In [17]:
weather.head(100)

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,631152000,1990-01-01 00:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,37.49,27.45,36.72,37.94,...,270,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
1,631155600,1990-01-01 01:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,37.65,27.63,37.24,37.94,...,270,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
2,631159200,1990-01-01 02:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,36.93,24.48,36.75,37.04,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
3,631162800,1990-01-01 03:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,36.81,24.33,36.38,37.04,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
4,631166400,1990-01-01 04:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,36.77,24.28,36.41,37.04,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,631483200,1990-01-04 20:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,59.95,50.90,58.66,60.98,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04d
96,631486800,1990-01-04 21:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,60.01,50.97,58.51,60.98,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04d
97,631490400,1990-01-04 22:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,56.70,46.99,51.48,60.98,...,290,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04d
98,631494000,1990-01-04 23:00:00 +0000 UTC,-21600,Nashville,36.162664,-86.781602,53.22,46.65,50.71,55.04,...,310,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n


In [24]:
#weather['dt_iso'] = pd.to_datetime(weather['dt_iso'])
weather['dt_iso']= pd.to_datetime(weather['dt_iso'].str.strip(),  format = "%Y-%m-%dT%H:%M:%S")

ValueError: time data 1990-01-01 00:00:00 +0000 UTC doesn't match format specified

In [29]:
#weather['dt_iso'] = weather['dt_iso'].str.replace('+0000 UTC', '')
weather['dt_iso'] = weather['dt_iso'].map(lambda x: str(x)[:-5])

In [34]:
weather.columns


Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [31]:
weather.to_csv('CorrectWeather.csv')

In [49]:
accidents.to_csv('CorrectAccidents.csv')